In [ ]:
import pandas as pd
import numpy as np
import torch
from gensim.models import Word2Vec
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#chessData = pd.read_csv('Data/chessData.csv')
#random_evals = pd.read_csv('Data/random_evals.csv')-
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [3]:
len(tactic_evals)
tactic_evals = tactic_evals[:500000]


In [ ]:
tactic_evals_check = tactic_evals.dropna()
tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))

predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
predictors_raw.head()

In [5]:
def separate_board(board):
    return board.replace('/', ' ').split()

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [6]:
tokens_lines = list(map(separate_board, np.asarray(tactic_evals_check['FEN'])))
tokens_lines = [line[0:8] for line in tokens_lines]

tokens_moves = list(map(separate_move, np.asarray(tactic_evals_check['Move'])))
#tokens_piece = list(map(fen_to_tokens_without_spaces, np.asarray(tactic_evals['FEN'])))
predictors_raw['FEN'] = tokens_lines
predictors_raw['Move'] = tokens_moves


In [ ]:
fen_embedding_dim = 32
fen_embeddings = Word2Vec(sentences=tokens_lines, vector_size=fen_embedding_dim, window=5, min_count=1, sg=1)
fen_embeddings.train(tokens_lines, total_examples=len(tokens_lines), epochs=10)

In [ ]:
move_embedding_dim = 32
move_embeddings = Word2Vec(sentences=tokens_moves, vector_size=move_embedding_dim, window=5, min_count=1, sg=1)
move_embeddings.train(tokens_moves, total_examples=len(tokens_moves), epochs=10)

In [9]:
fen_data = []
move_data = []

for line in range(len(tokens_lines)):
    fen_data.append(fen_embeddings.wv[tokens_lines[line]])

for move in range(len(tokens_moves)):
    move_data.append(move_embeddings.wv[tokens_moves[move]])

predictors = pd.DataFrame({
    'fen_data': fen_data,
    'move_data': move_data
})

In [13]:
move_data_expanded = []
for embedding in move_data:
    if embedding.shape == (4, 32):
        move_data_expanded.append(np.vstack([embedding, np.zeros((4, 32))]))
    else:
        move_data_expanded.append(np.vstack([embedding, np.zeros((3, 32))]))
        
predictors['move_data'] = move_data_expanded

In [ ]:
print(f"fen_data_shape: {predictors['fen_data'][0].shape}\n move_data_shape: {predictors['move_data'][0].shape}")


In [ ]:
fen_data = np.array(predictors['fen_data'].tolist())
move_data = np.array(predictors['move_data'].tolist()) 


print(fen_data.shape) 
print(move_data.shape) 


X_fen = np.reshape(fen_data, (fen_data.shape[0], 8, 32, 1)) 
X_move = np.reshape(move_data, (move_data.shape[0], 8, 32, 1)) 

y = tactic_evals_check['isCheck'].to_numpy()  


print(X_fen.shape) 
print(X_move.shape)  
print(y.shape)  

In [ ]:
predictors.head()

In [ ]:
ncols = predictors.shape[1]

fen_input = Input(shape=(8, 32, 1), name='fen_input')
move_input = Input(shape=(8, 32, 1), name='move_input')

fen_conv1 = Conv2D(16, (2,2), activation='relu')(fen_input)
move_conv1 = Conv2D(16, (2,2), activation='relu')(move_input)

fen_mp1 = MaxPooling2D(pool_size=(2,2))(fen_conv1)
move_mp1 = MaxPooling2D(pool_size=(2,2))(move_conv1)

fen_flatter = Flatten()(fen_mp1)
move_flatter = Flatten()(move_mp1)

combined = Concatenate()([fen_flatter, move_flatter])

dense = (Dense(32, activation = 'relu'))(combined)
output = (Dense(1 , activation = 'sigmoid'))(dense)

mate_evaluator = Model(inputs=[fen_input, move_input], outputs=output)
mate_evaluator.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
		      metrics=['accuracy'])

mate_evaluator.summary()

In [ ]:
#acc = 0.8509
early_stopping_monitor = EarlyStopping(patience=2)
mate_evaluator.fit([X_fen, X_move], y, 
                    validation_split=0.3, 
                    epochs=10, 
                    callbacks=[early_stopping_monitor])

In [ ]:
test_match_board = 'R3K1NR/PPPBBPPP/8/k7/3P2pp/p3P3/1pQ1p3/1r4nr'
test_match_mate = '2da5'

sep_board = separate_board(test_match_board)
sep_mate = separate_move(test_match_mate)


board_emb = fen_embeddings.wv[sep_board]
mate_emb = move_embeddings.wv[sep_mate]

print(board_emb.shape)
print(mate_emb.shape)

#mate_evaluator.predict([])